In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as sfs


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def sfsFeature(indep_x,dep_y):
    SFSlist=[]
    SFSlist1=[]
    SFSlist2=[]
     


    #log_model = LogisticRegression(solver='lbfgs')
    svc_model=SVC(kernel='linear',max_iter=100,random_state=0)
    #DT=DecisionTreeClassifier(criterion='gini',max_features='sqrt',splitter='best',random_state=0)
    #RF=RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=0)
    SFSmodellist=[svc_model]
    

    for model in SFSmodellist:
        log_sfs = sfs(model, k_features=6, forward=False, floating=False, verbose=0, scoring='accuracy', cv=5, n_jobs=1)
        log_fit = log_sfs.fit(indep_x, dep_y)
        log_sfs_feature = log_fit.transform(indep_x)
        SFSlist.append(log_sfs_feature)

        # Get the indices and names of selected features
        op = log_sfs.k_feature_idx_
        op1 = log_sfs.k_feature_names_
        SFSlist1.append(op)
        SFSlist2.append(op1)

    return SFSlist, SFSlist1, SFSlist2

In [4]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        
        return X_train, X_test, y_train, y_test, sc
    
def cm_prediction(classifier,X_test):
        y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(y_test, y_pred)
        
        from sklearn.metrics import accuracy_score 
        from sklearn.metrics import classification_report 
        
        Accuracy=accuracy_score(y_test, y_pred )
        
        report=classification_report(y_test, y_pred)
        return  classifier,Accuracy,report,X_test,y_test,cm


In [5]:
def random(X_train,y_train,X_test):
        
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm



def sfs_classification(accrf):
   
    sfsdataframe = pd.DataFrame(index=['SVC'],
                                 columns=[ 'Random'])
    for number,idex in enumerate(sfsdataframe.index):
        sfsdataframe['Random'][idex]=accrf[number]
    return sfsdataframe

In [6]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True)

indep_X=df2.drop('classification_yes', axis=1)
dep_Y=df2['classification_yes']

In [7]:
SFSlist=sfsFeature(indep_X,dep_Y)       


accrf=[]


In [8]:
SFSlist, SFSlist1,SFSlist2=SFSlist

In [9]:
SFSlist

[array([[ 3.        ,  3.07735602, 12.51815562,  1.        ,  0.        ,
          0.        ],
        [ 2.        ,  0.7       , 10.7       ,  1.        ,  0.        ,
          0.        ],
        [ 1.        ,  0.6       , 12.        ,  0.        ,  0.        ,
          0.        ],
        ...,
        [ 3.        ,  6.        ,  9.1       ,  1.        ,  0.        ,
          1.        ],
        [ 0.        ,  6.8       ,  8.5       ,  0.        ,  0.        ,
          1.        ],
        [ 0.        ,  1.        , 16.3       ,  0.        ,  0.        ,
          0.        ]])]

In [10]:
 SFSlist1

[(2, 6, 9, 14, 15, 22)]

In [11]:
SFSlist2

[('al', 'sc', 'hrmo', 'sg_c', 'sg_d', 'dm_yes')]

In [12]:
for i in SFSlist:   
    

    X_train, X_test, y_train, y_test,sc=split_scalar(i,dep_Y)   
    
    
    classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
    accrf.append(Accuracy)
    
    
result=sfs_classification(accrf)

In [13]:
result
#6

,Random
SVC,1


In [14]:
import pickle as pkl
filename="finalized_model_SVC.sav"

In [15]:
pickle.dump(classifier,open(filename,'wb'))

In [16]:
filenamex="sc.pkl"

In [17]:
pickle.dump(sc,open(filenamex,'wb'))

In [18]:
sc=pickle.load(open("sc.pkl",'rb'))

In [25]:
preinput = sc.transform([[0,0,16.7 ,50,0,1]])
preinput

array([[ -0.65587132,  -0.55748033,   1.55577448, 121.38194619,
         -0.46416285,   1.36930639]])

In [26]:
loaded_model=pickle.load(open("finalized_model_SVC.sav",'rb')) 
result = loaded_model.predict(preinput)

In [27]:
result

array([1], dtype=uint8)